# DATA RETRIEVAL WITH BGG API FOR IMAGES, DESCRIPTIONS & PRICES

## IMPORT LIBRARIES

In [1]:
import pandas as pd
import requests
import xmltodict
import random
import json
from IPython.display import clear_output
from xml.etree import ElementTree

## Functions

In [2]:
def extract_euro_data(data):
    # Initialize empty lists to store items with prices in different currencies
    euro_data = []  # For items with price in euros
    
    try:
        # Iterate through each item in the provided data list
        for item in data:
            # Check if the item's currency is 'EUR'
            if item['price']['@currency'] == 'EUR':
                # If the currency is 'EUR', add the item to the euro_data list
                euro_data.append(item)
    except TypeError:
        # This part handles the case when the 'data' itself is a single item, not a list of items
        # Check if the item's currency is 'EUR'
        if data['price']['@currency'] == 'EUR':
            # If the currency is 'EUR', assign the single item to the euro_data list
            euro_data = data
    
    if not euro_data:
        # If there are no items with price in euros, return 0
        return 0
    
    # Return the list of items with price in euros
    return euro_data

In [3]:
def get_data_bgg(sample):
    # Initialize empty lists to store the extracted data
    wait_time = 0.3  # A wait time in seconds between API requests (not used in this code)
    bgg_id = []  # List to store board game IDs
    image = []  # List to store board game images
    description = []  # List to store board game descriptions
    geekersprice = []  # List to store the most recent Geekers price of the board game
    currency = []  # List to store the currency of the most recent Geekers price
    listdate = []  # List to store the most recent list date of the board game
    link = []  # List to store the link of the most recent Geekers price listing
    a = 0  # Counter for tracking the progress while processing the board game IDs

    # Loop through each board game ID in the sample list
    for s in sample:
        clear_output(wait=True)  # Clear the output display to show the current progress
        print(a)  # Print the current progress (counter value)
        a += 1  # Increment the counter

        # Send a GET request to the specified URL to retrieve data in XML format
        response = requests.get(f'https://boardgamegeek.com/xmlapi/boardgame/{s}?marketplace=1')
        # Parse the XML response into a Python dictionary using xmltodict library
        # process_namespaces=True is set to process any XML namespaces if present
        dict_data = xmltodict.parse(response.content, process_namespaces=True)
        # Decode the response content from bytes to a UTF-8 string
        decoded_response = response.content.decode('utf-8')
        # Convert the decoded XML response into a JSON string and then parse it back into a Python dictionary
        # This step is mainly done to ensure compatibility and consistency when working with the data further
        response_json = json.loads(json.dumps(xmltodict.parse(decoded_response)))

        # Extract relevant data from the response and append it to the respective lists
        try:
            bgg_id.append(response_json['boardgames']['boardgame']['@objectid'])
            description.append(response_json['boardgames']['boardgame']['description'])
            image.append(response_json['boardgames']['boardgame']['image'])
        except KeyError:
            continue

        try:
            # Extract the most recent Euro data using the extract_euro_data function
            euro_data = extract_euro_data(response_json['boardgames']['boardgame']['marketplacelistings']['listing'])
            try:
                if euro_data != 0:  # If euro_data is not zero (i.e., there's Euro currency data available)
                    most_recent_listdate = max(euro_data, key=lambda x: x['listdate'])
                    geekersprice.append(most_recent_listdate.get('price').get("#text"))
                    currency.append(most_recent_listdate.get('price').get('@currency'))
                    listdate.append(most_recent_listdate.get('listdate'))
                    link.append(most_recent_listdate.get('link').get('@href'))
                else:  # If euro_data is zero (no Euro currency data available)
                    geekersprice.append(0.00)
                    currency.append('EUR')
                    listdate.append('')
                    link.append('')
            except TypeError:
                # Handle the case when there is only one listing, and extract_euro_data returns a dictionary
                most_recent_listdate = euro_data
                geekersprice.append(most_recent_listdate.get("#text"))
                currency.append(most_recent_listdate.get('price').get('@currency'))
                listdate.append(most_recent_listdate.get('price').get('listdate'))
                link.append(most_recent_listdate.get('link').get('@href'))
        except KeyError:
            # Handle the case when there is no marketplace data for the board game
            geekersprice.append(0.00)
            currency.append('EUR')
            listdate.append('')
            link.append('')

        # Create a pandas DataFrame using the extracted data
        df = pd.DataFrame(zip(bgg_id, image, description, geekersprice, currency, listdate, link))

    # Return the DataFrame containing the extracted data for all board game IDs
    return df

## Identify missing values

In [4]:
# Reads CSV File with games
bgames = pd.read_csv(r'DB/DB/bgames_v1.csv', low_memory=False)

In [5]:
# Identifies null values and creates a list called no_image that contains the ID of those games
no_image = list(bgames.loc[bgames.image.notnull()==False, 'bgg_id'])

In [6]:
# Creates a random sample
sample = random.sample(no_image,10)

## API

In [7]:
round(len(no_image)/10)

4956

In [8]:
df_5000=get_data_bgg(no_image[:5000])
df_5000.to_csv("DB/DB/df_5000.csv", index = False)

In [9]:
df_10000=get_data_bgg(no_image[5000:10000])
df_10000.to_csv("DB/DB/df_10000.csv", index = False)

In [10]:
df_15000=get_data_bgg(no_image[10000:15000])
df_15000.to_csv("DB/DB/df_15000.csv", index = False)

4999


In [10]:
df_20000=get_data_bgg(no_image[15000:20000])
df_20000.to_csv("DB/DB/df_20000.csv", index = False)

4999


In [11]:
df_25000=get_data_bgg(no_image[20000:25000])
df_25000.to_csv("DB/DB/df_25000.csv", index = False)

4999


In [12]:
df_30000=get_data_bgg(no_image[25000:30000])
df_30000.to_csv("DB/DB/df_30000.csv", index = False)

4999


In [13]:
df_35000=get_data_bgg(no_image[30000:35000])
df_35000.to_csv("DB/DB/df_35000.csv", index = False)

4999


In [14]:
df_40000=get_data_bgg(no_image[35000:40000])
df_40000.to_csv("DB/DB/df_40000.csv", index = False)

4999


In [15]:
df_45000=get_data_bgg(no_image[40000:45000])
df_45000.to_csv("DB/DB/df_45000.csv", index = False)

9561


In [ ]:
df_50000=get_data_bgg(no_image[45000:50000])
df_50000.to_csv("DB/DB/df_50000.csv", index = False)